In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Backtester Alpaca Broker

## Installs

In [ ]:
!pip install -qq alpaca-trade-api
!pip install -qq backtrader

     |████████████████████████████████| 103 kB 8.2 MB/s 
     |████████████████████████████████| 1.3 MB 31.6 MB/s 
     |████████████████████████████████| 636 kB 41.6 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 273 kB 67.8 MB/s 
     |████████████████████████████████| 160 kB 52.7 MB/s 
     |████████████████████████████████| 271 kB 47.8 MB/s 
     |████████████████████████████████| 410 kB 5.2 MB/s 


In [ ]:
!cd /content/gdrive/MyDrive/trading_bot

In [ ]:
import sys

if '/content/gdrive/MyDrive/trading_bot' not in sys.path:
  sys.path.append('/content/gdrive/MyDrive/trading_bot') #Agrego el path de las credenciales

##Imports de Alpaca y Credenciales

In [ ]:
import numpy as np
import backtrader as bt
import alpaca_trade_api as tradeapi
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

##Implementacion

###Metricas

In [ ]:
def Metricas(data_dict, TP, SL):
  
  #Iinicializo una memoria

  memory = {'datetime' : [],
            'buy' : [],
            'sell': []
            }


  #Inicializo las metricas
  resultado_total = 0
  efectividad_total = 0
  

  for date, prices in data_dict.items():
    
    #Resultado: 
    #porcentaje de variacion al vender respecto de la compra
    resultado = (prices[1] / prices[0]) - 1
    resultado_total += resultado
    #Efectividad
    #Cantidad de operaciones + respecto del total
    efectividad = int(prices[1] > prices[0])
    efectividad_total += efectividad


    
    memory['datetime'].append(date)
    memory['buy'].append(prices[0])
    memory['sell'].append(prices[1])



  if len(data_dict) != 0:
    resultado_total = resultado_total
    efectividad_total = efectividad_total/len(data_dict)
  else:
    resultado_total = np.NaN
    efectividad_total = np.NaN

  operaciones = len(data_dict)
  print(f"Resultado total: {100*resultado_total:.2f}%")
  print(f"Efectividad total: {100*efectividad_total:.2f}%")
  print(f"RR: {abs(TP/SL):.2f}")

  return np.round(resultado_total*100, 3), np.round(efectividad_total*100, 3), abs(TP/SL), memory, operaciones

## Credenciales


In [ ]:
from alpaca_trade_api.rest import REST, TimeFrame


ALPACA_KEY_ID = ''
ALPACA_SECRET_KEY = ''

ALPACA_PAPER = True
base_url = 'https://paper-api.alpaca.markets'

api = REST(ALPACA_KEY_ID, ALPACA_SECRET_KEY, base_url, api_version='v2')

print('API STATUS:', api.get_account().status)


API STATUS: ACTIVE


###Estrategia

In [ ]:
memory_precios_OpenClose = {}
#@title
class Strategy_v1(bt.Strategy):
    params = (
        ('TP', None),
        ('SL', None),
        ('Trail', None),
        ('startDate', None)
    )

    def __init__(self):
        self.current_day = self.params.startDate
        
        #Moving averages
        self.Vol_SMA = bt.ind.SimpleMovingAverage(self.data.volume,period=21)
        self.Price_SMA = bt.ind.SimpleMovingAverage(self.data.close,period=3)

        #Inicializo parametros para el calculo de los filtros
        # Contador diario de velas:
        self.n_vela = 1
        #Rendimiento de la primer vela:
        self.rend_1erVela = 0
        #Gap
        self.gap = 1
        #Cierre de ayer para calculo del rendimiento
        self.close_ayer = 0
        self.dt = None

        #Seteo TP, SL y Trail:
        self.TP = self.params.TP
        self.SL = self.params.SL
        self.Trail = self.params.Trail 
        self.trigger_trail = True
        self.today = 'Open'
        self.close_ayer = 58

    def log(self, txt, dt=None, tt=None):
        if dt == None:
          dt = self.datas[0].datetime.date(0)
        if tt == None:
          tt = self.datas[0].datetime.time(0)
        print('%s, %s, %s' % (dt.isoformat(), tt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
              self.log(f'BUY EXECUTED {order.executed.price:.2f} TP: {self.TP_price:.2f} SL: {self.SL_price:.2f}')
              self.dt  = self.datas[0].datetime.date(0).strftime("%Y-%m-%d")
              memory_precios_OpenClose[stock][estrategia][self.dt] = [order.executed.price]
            elif order.issell():
              self.log('SELL EXECUTED, %.2f' % order.executed.price)
              self.dt = self.datas[0].datetime.date(0).strftime("%Y-%m-%d")
              memory_precios_OpenClose[stock][estrategia][self.dt].append(order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f' %
                 (trade.pnl))


    def next(self):       
      #Lo siguiente se ejecuta en la primera vela del dia
      # if self.datas[0].datetime.date(0).day != self.current_day:
      if self.datas[0].datetime.time(0).hour == 15 and self.datas[0].datetime.time(0).minute == 57:
        #Cierre de ayer para calculo del rendimiento
        self.close_ayer = 1*self.datas[0].close
        # self.log("cierre ayer: " + str(self.close_ayer))
        
      if self.datas[0].datetime.time(0).hour == 9 and self.datas[0].datetime.time(0).minute == 30:
        self.today = 'Open'
        self.log('OPEN DAY \n' + "Close: " + str(1*self.datas[0].close) + " Open: " + str(1*self.datas[0].open))
        #Reseteo los contadores
        self.n_vela = 0
        # self.log(self.n_vela)
        self.current_day = self.datas[0].datetime.date(0).day

        #Rendimiento de la primer vela:
        self.rend_1erVela = (self.datas[0].close / self.datas[0].open) - 1
        # self.log("Close: " + str(1*self.datas[0].close) + " Open: " + str(1*self.datas[0].open))
        #Gap
        self.gap = (self.datas[0].open / self.close_ayer) - 1
        

      #Si estamos entre la 2da y 5ta vela podemos entrar
      #El limite de 5ta vela es relajable!
      elif self.n_vela < 6 and not self.position and self.today == 'Open':
          #self.log(self.n_vela)
          #Inicializo la lista de filtros
          filtros = ['']*6

          #Media movil de 3 periodos
          filtros[0] = (np.floor(1*self.datas[0].low)) <= (1*self.Price_SMA) #and (self.Price_SMA <= self.datas[0].high)
          
          #No retroceso
          filtros[1] = (np.round(1*self.datas[0].open, 1)) >= (1*self.datas[-1].close)

          #Gap < 1 %
          filtros[2] = np.round(1*self.gap,2) <= 0.01
          
          #Rend 1era vela > 1%
          filtros[3] = np.round(1*self.rend_1erVela, 2) >= 0.01
          # self.log("Rendimiento 1era_vela " + str(1*self.rend_1erVela))
          
          #Rendimiento entre 1% - 5%
          self.rendimiento = (self.datas[0].close / self.close_ayer) - 1
          self.rendimiento = np.round(1*self.rendimiento, 2)
          
          if self.rendimiento >= 0.01 and self.rendimiento <= 0.05:
            filtros[4] = True
          else:
            filtros[4] = False
          
          #Volumen
          filtros[5] = (1*self.datas[0].volume) >= (self.Vol_SMA * 0.99)

          # print(['3 periodos', 'retroceso', 'gap', 'primera vela','rendimiento','volumen'])
          # print(filtros)

          #Si cumple con los requisitos (TODOS) entonces compra
          if all(filtros):
            self.market = False
            precio = self.Price_SMA * 1.002
            self.buy(price=precio)
            memory_trade_type[stock][estrategia][str(self.datas[0].datetime.datetime(0))] = 'buy'
            self.TP_price = precio * ( 1 + self.TP)
            self.SL_price = precio * ( 1 - self.SL)
            self.ST_price = 0
            self.SL_per = self.Trail * (1 - 0.3)
            self.Trail_price = precio * ( 1 + self.Trail)
        
      #Si ya compro ejecuta los controles
      if self.position:
        #Controlar

        if self.datas[0].close >= self.Trail_price and self.trigger_trail == True:
            self.ST_price = self.Trail_price * ( 1 - self.SL_per)
            self.last_Trail_price = 1*self.Trail_price
            self.trigger_trail = False

        elif self.trigger_trail == False and self.datas[0].close > self.last_Trail_price:
          self.last_Trail_price = 1*self.datas[0].close
          self.ST_price = self.last_Trail_price * ( 1 - self.SL_per)

        #Vende si supera take profit
        elif self.datas[0].close >= self.TP_price:
          print("Take Profit")
          self.order = self.sell(price=self.TP_price)
          self.today = 'Close'
          memory_trade_type[stock][estrategia][str(self.datas[0].datetime.datetime(0))] = 'Take Profit'

        elif self.datas[0].close <= self.SL_price:
          print("Stop Loss")
          self.order = self.sell(price=self.SL_price)
          self.today = 'Close'
          memory_trade_type[stock][estrategia][str(self.datas[0].datetime.datetime(0))] = 'Stop Loss'

        elif self.datas[0].close <= self.ST_price:
          print("Stop trail Loss")
          self.order = self.sell(price=self.ST_price)
          self.today = 'Close'
          memory_trade_type[stock][estrategia][str(self.datas[0].datetime.datetime(0))] = 'Stop trail Loss'

        #Al final del dia vende (a las 15:59)
        #Se implemento en la primer vela del dia siguiente usando el indeice -1
        if self.datas[0].datetime.time(0).hour == 15 and self.datas[0].datetime.time(0).minute > 53:
          self.log("Cierre del dia")
          print("market")
          self.sell()
          self.today = 'Close'
          memory_trade_type[stock][estrategia][str(self.datas[0].datetime.datetime(0))] = 'market'

      #final de la vela
      self.n_vela += 1

###Loop

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/trading_bot/estrategias.csv')

df = df.groupby(['accion','estrategia']).first().copy()

df.head()

TP    SL     TR  sample  sample_per
accion estrategia                                         
AMAT   estrategia_0  2.33 -1.79   1.49     198       15.74
       estrategia_1  1.02 -1.93   0.79     239       19.00
       estrategia_2  0.98 -1.91   0.63      21       18.10
AMC    estrategia_0  9.33 -6.70  11.68      25        1.99
       estrategia_1  2.39 -2.58   1.48     226       17.97

In [ ]:
Stocks = set(list(df.index.get_level_values(0)))

#Stocks

In [ ]:
summary = {}
memory_trade_type = {}
memory_precios_OpenClose = {}
memory_trade_results = {}

for stock in Stocks:
  print(stock)
  #Cargo los parametros
  records = df.loc[stock,['TP','SL','TR']].to_records(index=False)
  parameters = list(records)
  # parameters = [[1.96, 1.60,	1.17]]

  print(f'Stock:    TP      SL    TR ')
  print(f'        {parameters[0]:}')
  print(f'        {parameters[1]:}')
  print(f'        {parameters[2]:}')

  summary[stock] = {'estrategia_0': {},'estrategia_1': {},'estrategia_2': {},}

  estrategia = 0
  memory_trade_type[stock] = {}
  memory_precios_OpenClose[stock] = {}
  memory_precios_OpenClose[stock] = {}
  memory_trade_results[stock] = {}
  for params in parameters:
    print(f'Estrategia: {estrategia}')
    memory_trade_type[stock][estrategia] = {}
    memory_precios_OpenClose[stock][estrategia] = {}
    memory_trade_results[stock][estrategia] = {}
    

    dataframe = api.get_bars(stock, "1Min", "2021-08-07", "2021-11-09", adjustment='raw').df
    #Elimino columnas que sobran
    dataframe = dataframe.drop(columns = ['trade_count', 'vwap'])

    #Renombro el indice
    dataframe.index = dataframe.index.tz_convert('America/New_York')
    #Filtro por horario
    dataframe = dataframe.between_time('9:30','15:59')
    dataframe.index = pd.DatetimeIndex([i.replace(tzinfo=None) for i in dataframe.index])
    dataframe.index.name = 'datetime'


    #Seteo el indice (timestamp) como columna
    dataframe.reset_index(inplace = True)
    dataframe = dataframe.groupby(pd.Grouper(key='datetime', freq='3min')).agg({'open' : 'first', 'close':'last', 'high':'max', 'low': 'min', 'volume':'sum'})
    dataframe = dataframe.between_time('9:30','15:59')
    dataframe.reset_index(inplace = True)

    # saco los fines de semana
    dataframe = dataframe.loc[dataframe.datetime.dt.dayofweek < 5, :]

    data0 = bt.feeds.PandasData(dataname=dataframe,
                              datetime=0,
                              high=3,
                              low=4,
                              open=1,
                              close=2,
                              volume=5,
                              openinterest=-1)

    #Instantiate Cerebro engine
    cerebro = bt.Cerebro()

    cerebro.adddata(data0)

    TP_ = params[0]/100
    SL_ = -params[1]/100  # le agrego signo negativo
    Trail_ = params[2]/100

    cerebro.addstrategy(Strategy_v1, TP = TP_,SL = SL_, Trail = Trail_, startDate=None)
    #Run Cerebro Engine
    cerebro.run()
    Resultados, Efectividad, RR, _, operaciones = Metricas(memory_precios_OpenClose[stock][estrategia], TP = params[0], SL = params[1])

    summary[stock][f'estrategia_{estrategia}']['Resultados'] = Resultados
    summary[stock][f'estrategia_{estrategia}']['Efectividad'] = Efectividad
    summary[stock][f'estrategia_{estrategia}']['RR'] = RR
    summary[stock][f'estrategia_{estrategia}']['Operaciones'] = operaciones

    memory_trade_results[stock][estrategia] = _

    #print(memory_trade_type[stock][estrategia])
    #print(memory_precios_OpenClose[stock][estrategia])
    estrategia += 1

In [ ]:
#memory_trade_results

In [ ]:
summary

In [ ]:
dummy_df = pd.DataFrame.from_dict({(i,j): summary[i][j] 
                            for i in summary.keys() 
                            for j in summary[i].keys()},
                            orient='index')

dummy_df.index = pd.MultiIndex.from_tuples(dummy_df.index, names=('accion',	'estrategia'))

df_final = df.merge(dummy_df, left_index=True, right_index=True, how='outer')

df_final

TP    SL     TR  ...  Efectividad        RR  Operaciones
accion estrategia                       ...                                    
AMAT   estrategia_0  2.33 -1.79   1.49  ...       33.333  1.301676            3
       estrategia_1  1.02 -1.93   0.79  ...       33.333  0.528497            3
       estrategia_2  0.98 -1.91   0.63  ...       66.667  0.513089            3
AMC    estrategia_0  9.33 -6.70  11.68  ...       25.000  1.392537            8
       estrategia_1  2.39 -2.58   1.48  ...       25.000  0.926357            8
...                   ...   ...    ...  ...          ...       ...          ...
XOP    estrategia_1  0.93 -1.89   0.76  ...       57.143  0.492063            7
       estrategia_2  1.96 -2.13   1.00  ...       71.429  0.920188            7
XPEV   estrategia_0  4.08 -3.57   3.28  ...       40.000  1.142857           10
       estrategia_1  4.41 -3.67   1.94  ...       50.000  1.201635           10
       estrategia_2  4.83 -2.25   2.03  ...       50.000  2.146667           10

[120 rows x 9 columns]

In [ ]:
#df_final.loc['RIOT']

In [ ]:
df_final.to_csv('/content/gdrive/MyDrive/trading_bot/summary.csv')

In [ ]:
#df_ = pd.read_csv('/content/gdrive/MyDrive/trading_bot/summary.csv')
#df_.groupby(['accion','estrategia']).first().loc['AMD']

In [ ]:
import json

with open('/content/gdrive/MyDrive/trading_bot/save_precios.json', 'w') as fp:
    json.dump(memory_precios_OpenClose, fp)

with open('/content/gdrive/MyDrive/trading_bot/save_tipos.json', 'w') as fp:
    json.dump(memory_trade_type, fp)

with open('/content/gdrive/MyDrive/trading_bot/save_results.json', 'w') as fp:
    json.dump(memory_trade_results, fp)

In [ ]:
df_trades = pd.DataFrame.from_dict({(i,j,k): memory_trade_type[i][j][k] 
                            for i in memory_trade_type.keys() 
                            for j in memory_trade_type[i].keys()
                            for k in memory_trade_type[i][j].keys()},
                            orient='index')

df_trades.index = pd.MultiIndex.from_tuples(df_trades.index, names=('accion',	'estrategia', 'datetime'))

for i in range(len(df_trades)):
  if 'Stop Loss' in df_trades.iloc[i,0]:
    df_trades.iloc[i,0] = 'Stop Loss'

df_trades.columns = ['Type']

df_trades.reset_index(2,inplace=True)

#df_trades.to_csv('/content/gdrive/MyDrive/trading_bot/Trades.csv')

df_trades = df_trades[df_trades.Type != 'buy']
df_trades['datetime'] = df_trades.apply(lambda x: x['datetime'].split()[0], axis=1)
df_trades

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


datetime             Type
accion estrategia                             
DAL    0           2021-08-10      Take Profit
       0           2021-08-31  Stop trail Loss
       0           2021-09-09      Take Profit
       0           2021-09-16           market
       0           2021-09-17        Stop Loss
...                       ...              ...
PFE    1           2021-10-19      Take Profit
       2           2021-08-13      Take Profit
       2           2021-10-05           market
       2           2021-10-14      Take Profit
       2           2021-10-19           market

[780 rows x 2 columns]

In [ ]:
df_trades_results = pd.DataFrame.from_dict({(i,j): memory_trade_results[i][j] 
                            for i in memory_trade_results.keys() 
                            for j in memory_trade_results[i].keys()},
                            orient='index')

df_trades_results.index = pd.MultiIndex.from_tuples(df_trades_results.index, names=('accion',	'estrategia'))


df_trades_results = df_trades_results.apply(pd.Series.explode)

df_trades_results

datetime    buy    sell
accion estrategia                           
DAL    0           2021-08-10  39.63   40.56
       0           2021-08-31  40.16   40.23
       0           2021-09-09  40.17   41.38
       0           2021-09-16  39.87  40.205
       0           2021-09-17  40.62   39.92
...                       ...    ...     ...
PFE    1           2021-10-19  41.97  42.175
       2           2021-08-13  47.97  48.255
       2           2021-10-05  42.68   42.31
       2           2021-10-14  41.74   41.92
       2           2021-10-19  41.97  42.055

[780 rows x 3 columns]

In [ ]:
df_trades_final = df_trades.merge(df_trades_results, on=['accion', 'estrategia','datetime'], how='inner')

df_trades_final['Resultado'] = df_trades_final.apply(lambda x : np.round(((x['sell'] / x['buy']) - 1) * 100,2), axis= 1)

df_trades_final.to_csv('/content/gdrive/MyDrive/trading_bot/trades.csv')

df_trades_final

datetime             Type    buy    sell  Resultado
accion estrategia                                                       
DAL    0           2021-08-10      Take Profit  39.63   40.56       2.35
       0           2021-08-31  Stop trail Loss  40.16   40.23       0.17
       0           2021-09-09      Take Profit  40.17   41.38       3.01
       0           2021-09-16           market  39.87  40.205       0.84
       0           2021-09-17        Stop Loss  40.62   39.92      -1.72
...                       ...              ...    ...     ...        ...
PFE    1           2021-10-19      Take Profit  41.97  42.175       0.49
       2           2021-08-13      Take Profit  47.97  48.255       0.59
       2           2021-10-05           market  42.68   42.31      -0.87
       2           2021-10-14      Take Profit  41.74   41.92       0.43
       2           2021-10-19           market  41.97  42.055       0.20

[780 rows x 5 columns]